In [10]:
from model.stconvs2s import STConvS2S_R
 
from tool.train_evaluate import Trainer, Evaluator
from tool.dataset import CORDataset
from tool.utils import Util

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

import h5py

import numpy as np

In [11]:
file = './data/DataseParaWorkshop_v2.h5'

In [12]:
with h5py.File(file, 'r') as f:
        dados_treino = f["Treino"][...]
        dados_validacao = f["Validacao"][...]
        dados_teste = f["Teste"][...]

### 10 dias em que mais choveu

In [28]:
dados_teste.shape

(2, 849, 7, 7)

In [29]:
maximos = np.amax(dados_teste, axis=(0,2,3))

In [30]:
idx_max = maximos.argsort()[-10:]

In [31]:
for idx in idx_max:
    print(maximos[idx])

15.550242201290231
16.209819161800887
16.53780709891247
17.23751285200541
19.347807917897565
20.297321356909993
23.75462182969954
23.759927126382053
24.474535836301037
27.59769060304221


In [32]:
idx_max

array([ 13, 525, 526, 290,  69, 287, 712, 288,  70, 521])

### 10 maiores eventos de chuva

In [33]:
max_indices = np.argsort(dados_teste.ravel())[-10:]

In [34]:
max_indices

array([25550, 25558, 25557, 25559, 25565, 25564, 25571, 25572, 25566,
       25573])

In [35]:
max_indices_completos = []
for idx in max_indices:
    idx_completo = np.unravel_index(idx, dados_teste.shape)
    max_indices_completos.append(idx_completo)
    

In [36]:
max_indices_completos

[(0, 521, 3, 0),
 (0, 521, 4, 1),
 (0, 521, 4, 0),
 (0, 521, 4, 2),
 (0, 521, 5, 1),
 (0, 521, 5, 0),
 (0, 521, 6, 0),
 (0, 521, 6, 1),
 (0, 521, 5, 2),
 (0, 521, 6, 2)]

In [37]:
np.unravel_index(14147, dados_teste.shape)

(0, 288, 5, 0)

In [39]:
dados_teste[0, 521, 6, 2]

27.59769060304221

### Recuperando modelo treinado

In [24]:
filename = './output/full-dataset/checkpoints/stconvs2s-r/cor_4_20211128-104224.pth.tar'

In [25]:
model = STConvS2S_R(input_size=(1, 2, 10, 7, 7), num_layers=3, hidden_dim=64, 
                    kernel_size=3, device=torch.device('cuda'), dropout_rate=0.0)

In [26]:
checkpoint = torch.load(filename)
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

### Normalizando os dados

In [40]:
train_min = np.amin(dados_treino, axis=(1, 2, 3)).reshape([2,1,1,1])
train_max = np.amax(dados_treino, axis=(1, 2, 3)).reshape([2,1,1,1])

In [41]:
dados_teste = (dados_teste - train_min) / (train_max - train_min)

In [42]:
train_min_tensor = torch.from_numpy(train_min).float().reshape(1, 2, 1, 1, 1)
train_max_tensor = torch.from_numpy(train_max).float().reshape(1, 2, 1, 1, 1)

### Previsão 10 maiores eventos de chuva

In [43]:
# todos ocorreram no exemplo 521, então esse é o exemplo que fica no y. No x ficam os 10 anteriores

In [44]:
y_inicial = 521
y_final = y_inicial + 1
x_inicial = y_inicial - 10
x_final = y_inicial

In [45]:
x = dados_teste[:, x_inicial:x_final]
y = dados_teste[:, y_inicial:y_final]

In [46]:
x = torch.tensor(x[None]).float()
y = torch.tensor(y[None]).float()

In [50]:
model.eval()
x.shape, y.shape

(torch.Size([1, 2, 10, 7, 7]), torch.Size([1, 2, 1, 7, 7]))

In [54]:
previsao = model(x)[0, :, -2:]

In [55]:
previsao.shape

torch.Size([2, 2, 7, 7])

In [50]:
# desnormalizacao
previsao = ( previsao * (train_max_tensor - train_min_tensor) ) + train_min_tensor
y = ( y * (train_max_tensor - train_min_tensor) ) + train_min_tensor

In [51]:
y_1d = y.flatten()
previsao_1d = previsao.flatten()

In [52]:
idx = torch.argsort(y_1d)[-10:]

In [53]:
y_final = torch.index_select(y_1d, 0, idx)
previsao_final = torch.index_select(previsao_1d, 0, idx)

In [54]:
y_final

tensor([26.6439, 26.7130, 26.8241, 26.9151, 27.2770, 27.3405, 27.3843, 27.4181,
        27.5379, 27.5977])

In [55]:
previsao_final

tensor([13.7475, 19.6509, 14.2838, 19.2150, 18.7547, 13.3748,  9.5744, 13.4647,
        18.8672, 13.7656], grad_fn=<IndexSelectBackward>)

In [56]:
mse = nn.MSELoss()

In [57]:
mse(previsao_final, y_final)

tensor(147.8807, grad_fn=<MseLossBackward>)

### Previsão 10 dias em que mais choveu

In [35]:
x, y = [], []
for idx in idx_max:
    y_inicial, y_final = idx, idx + 1
    x_inicial, x_final = idx - 10, idx
    x.append(torch.tensor(dados_teste[:, x_inicial:x_final]).float())
    y.append(torch.tensor(dados_teste[:, y_inicial:y_final]).float())

In [36]:
x = torch.stack(x)
x.shape

torch.Size([10, 2, 10, 7, 7])

In [37]:
y = torch.stack(y)
y.shape

torch.Size([10, 2, 1, 7, 7])

In [38]:
previsoes = model(x)[:, :, -2:-1]
previsoes.shape

torch.Size([10, 2, 1, 7, 7])

In [39]:
# desnormalizacao
previsoes = ( previsoes * (train_max_tensor - train_min_tensor) ) + train_min_tensor
y = ( y * (train_max_tensor - train_min_tensor) ) + train_min_tensor

In [40]:
mse(previsoes, y)

tensor(22.2007, grad_fn=<MseLossBackward>)